In [1]:
from pathlib import Path
import pandas as pd
pd.options.mode.copy_on_write = True
pd.set_option('display.max_colwidth', 20)   # JSON columns make things look weird in notebook without this

from datatype_recovery.models.dataset import load_dataset_from_path
from datatype_recovery.models.dataset.encoding import *

In [2]:
dataset_path = Path.home()/'datasets/coreutils_full_5hops'
dataset = load_dataset_from_path(dataset_path)

In [3]:
df = dataset.read_vars_csv()
df

,FunctionStart,Signature,Name_Strip,Type_Strip,LocType_Strip,LocRegName_Strip,LocOffset_Strip,TypeCategory_Strip,TypeSeq_Strip,Name_Debug,...,LocRegName_Debug,LocOffset_Debug,TypeCategory_Debug,TypeSeq_Debug,HasDWARF,TypeJson_Debug,BinaryId,Vartype,IsReturnType_Debug,IsReturnType_Strip
0,1056281,"37,78,82,107,111",local_a8,rusage,stack,NaN,-168.0,STRUCT,STRUCT,usage,...,NaN,-168.0,STRUCT,STRUCT,True,"{""kind"": ""Struct...",1000,l,False,False
1,1056439,"37,94,114",local_b8,timeval,stack,NaN,-184.0,STRUCT,STRUCT,real_end,...,NaN,-184.0,STRUCT,STRUCT,True,"{""kind"": ""Struct...",1000,l,False,False
2,1056439,"57,160,180,226,246",local_a8,rusage,stack,NaN,-168.0,STRUCT,STRUCT,usage,...,NaN,-168.0,STRUCT,STRUCT,True,"{""kind"": ""Struct...",1000,l,False,False
3,1056881,"300,300,300",auVar1,uchar[16],unique,NaN,243584.0,ARR,"ARR,uchar",auVar2,...,NaN,243584.0,ARR,"ARR,uchar",False,"{""kind"": ""ArrayT...",1000,l,False,False
4,1056881,"145,485",iVar2,int32,register,eax,0.0,BUILTIN,int32,iVar3,...,eax,0.0,BUILTIN,int32,False,"{""kind"": ""Builti...",1000,l,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85154,1092028,-1,NaN,uint32,NaN,NaN,NaN,BUILTIN,uint32,NaN,...,NaN,NaN,BUILTIN,char,True,"{""kind"": ""Builti...",1116,p,True,True
85155,1092192,-1,NaN,char*,NaN,NaN,NaN,PTR,"PTR,char",NaN,...,NaN,NaN,PTR,"PTR,char",True,"{""kind"": ""Pointe...",1116,p,True,True
85156,1092232,-1,NaN,uint64,NaN,NaN,NaN,BUILTIN,uint64,NaN,...,NaN,NaN,BUILTIN,int32,True,"{""kind"": ""Builti...",1116,p,True,True
85157,1092416,-1,NaN,void,NaN,NaN,NaN,BUILTIN,void,NaN,...,NaN,NaN,BUILTIN,int32,True,"{""kind"": ""Builti...",1116,p,True,True


In [4]:
df.TypeJson_Debug

dt = datatype_from_json_str(df.TypeJson_Debug.iloc[3])
dt.type_sequence[-1].category

# TODO: category
# - PRIMITIVE, STRUCT, UNION, FUNC[, ENUM?]
# TODO: size
# - 0, 1, 2, 4, 8[, 10?], 16[, >16?]
# --> long double (10) can be represented as 16 since we have no 16B floating point
# --> >16 can be represented as 16
# TODO: floating (0 or 1)
# TODO: signed (0 or 1)


'BUILTIN'

In [5]:
df = df.loc[df.TypeCategory_Debug!='COMP',:]

In [6]:
df['DebugType'] = df[(df.TypeCategory_Debug!='COMP')].TypeJson_Debug.apply(lambda x: datatype_from_json_str(x))

In [10]:
df.DebugType.iloc[0].leaf_type.category
df['LeafCategory'] = df.DebugType.apply(lambda x: x.leaf_type.category)
df['LeafSigned'] = df.DebugType.apply(lambda x: x.leaf_type.is_signed)
df['LeafFloating'] = df.DebugType.apply(lambda x: x.leaf_type.is_floating)
df['LeafSize'] = df.DebugType.apply(lambda x: x.leaf_type.primitive_size)

In [8]:
df.groupby('LeafCategory').count()

,FunctionStart,Signature,Name_Strip,Type_Strip,LocType_Strip,LocRegName_Strip,LocOffset_Strip,TypeCategory_Strip,TypeSeq_Strip,Name_Debug,...,TypeSeq_Debug,HasDWARF,TypeJson_Debug,BinaryId,Vartype,IsReturnType_Debug,IsReturnType_Strip,DebugType,LeafSigned,LeafFloating
LeafCategory,,,,,,,,,,,,,,,,,,,,,
BUILTIN,56366,56366,41369,56366,41369,26888,41369,56366,56366,41369,...,56366,56366,56366,56366,56366,56366,56366,56366,56366,56366
ENUM,2329,2329,1640,2329,1640,1129,1640,2329,2329,1640,...,2329,2329,2329,2329,2329,2329,2329,2329,2329,2329
FUNC,256,256,256,256,256,184,256,256,256,256,...,256,256,256,256,256,256,256,256,256,256
STRUCT,10327,10327,9589,10327,9589,6206,9589,10327,10327,9589,...,10327,10327,10327,10327,10327,10327,10327,10327,10327,10327
UNION,30,30,30,30,30,5,30,30,30,30,...,30,30,30,30,30,30,30,30,30,30


In [11]:
df.groupby('LeafSize').count()

,FunctionStart,Signature,Name_Strip,Type_Strip,LocType_Strip,LocRegName_Strip,LocOffset_Strip,TypeCategory_Strip,TypeSeq_Strip,Name_Debug,...,HasDWARF,TypeJson_Debug,BinaryId,Vartype,IsReturnType_Debug,IsReturnType_Strip,DebugType,LeafCategory,LeafSigned,LeafFloating
LeafSize,,,,,,,,,,,,,,,,,,,,,
0,19548,19548,13511,19548,13511,9259,13511,19548,19548,13511,...,19548,19548,19548,19548,19548,19548,19548,19548,19548,19548
1,20710,20710,14527,20710,14527,8522,14527,20710,20710,14527,...,20710,20710,20710,20710,20710,20710,20710,20710,20710,20710
2,356,356,298,356,298,252,298,356,356,298,...,356,356,356,356,356,356,356,356,356,356
3,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,15627,15627,12812,15627,12812,9428,12812,15627,15627,12812,...,15627,15627,15627,15627,15627,15627,15627,15627,15627,15627
6,3,3,3,3,3,0,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
8,12994,12994,11681,12994,11681,6916,11681,12994,12994,11681,...,12994,12994,12994,12994,12994,12994,12994,12994,12994,12994
16,69,69,51,69,51,34,51,69,69,51,...,69,69,69,69,69,69,69,69,69,69


In [15]:
print(hex(1069857))
df[df.LeafSize==3]

0x105321


,FunctionStart,Signature,Name_Strip,Type_Strip,LocType_Strip,LocRegName_Strip,LocOffset_Strip,TypeCategory_Strip,TypeSeq_Strip,Name_Debug,...,TypeJson_Debug,BinaryId,Vartype,IsReturnType_Debug,IsReturnType_Strip,DebugType,LeafCategory,LeafSigned,LeafFloating,LeafSize
14088,1069857,"399,408,498",puVar5,uchar*,register,rdx,16.0,PTR,"PTR,uchar",puVar5,...,"{""kind"": ""Pointe...",1043,l,False,False,uint32*,BUILTIN,False,False,3


In [14]:
bdf = dataset.read_binaries_csv()
bdf[bdf.BinaryId==1043]

,BinaryId,Name,RunGid,OrigBinaryId
43,1043,who,0,43


In [42]:
df.groupby(['LeafCategory','LeafSigned','LeafFloating']).count()

FunctionStart  Signature  Name_Strip   
LeafCategory LeafSigned LeafFloating                                         
BUILTIN      False      False                 20105      20105       14232  \
             True       False                 35977      35977       26892   
                        True                    284        284         245   
ENUM         False      False                  2329       2329        1640   
FUNC         False      False                   256        256         256   
STRUCT       False      False                 10327      10327        9589   
UNION        False      False                    30         30          30   

                                      Type_Strip  LocType_Strip   
LeafCategory LeafSigned LeafFloating                              
BUILTIN      False      False              20105          14232  \
             True       False              35977          26892   
                        True                 284            245   
ENUM         False      False               2329           1640   
FUNC         False      False                256            256   
STRUCT       False      False              10327           9589   
UNION        False      False                 30             30   

                                      LocRegName_Strip  LocOffset_Strip   
LeafCategory LeafSigned LeafFloating                                      
BUILTIN      False      False                     8444            14232  \
             True       False                    18237            26892   
                        True                       207              245   
ENUM         False      False                     1129             1640   
FUNC         False      False                      184              256   
STRUCT       False      False                     6206             9589   
UNION        False      False                        5               30   

                                      TypeCategory_Strip  TypeSeq_Strip   
LeafCategory LeafSigned LeafFloating                                      
BUILTIN      False      False                      20105          20105  \
             True       False                      35977          35977   
                        True                         284            284   
ENUM         False      False                       2329           2329   
FUNC         False      False                        256            256   
STRUCT       False      False                      10327          10327   
UNION        False      False                         30             30   

                                      Name_Debug  ...  LocOffset_Debug   
LeafCategory LeafSigned LeafFloating              ...                    
BUILTIN      False      False              14232  ...            14232  \
             True       False              26892  ...            26892   
                        True                 245  ...              245   
ENUM         False      False               1640  ...             1640   
FUNC         False      False                256  ...              256   
STRUCT       False      False               9589  ...             9589   
UNION        False      False                 30  ...               30   

                                      TypeCategory_Debug  TypeSeq_Debug   
LeafCategory LeafSigned LeafFloating                                      
BUILTIN      False      False                      20105          20105  \
             True       False                      35977          35977   
                        True                         284            284   
ENUM         False      False                       2329           2329   
FUNC         False      False                        256            256   
STRUCT       False      False                      10327          10327   
UNION        False      False                         30             30   

                                    

In [59]:
df['BuiltinSizes'] = df[(df.TypeCategory_Debug=='BUILTIN')].DebugType.apply(lambda x: x.size)

In [61]:
df.groupby('BuiltinSizes').count()[['FunctionStart']]

,FunctionStart
BuiltinSizes,
0.0,3444
1.0,6569
2.0,125
4.0,13119
6.0,3
8.0,12091
10.0,63
16.0,1
